In [1]:
import numpy as np 
import pandas as pd
from scipy.stats import norm

In [2]:
data = pd.read_csv("MSFT_AAPL_Log_Returns_20110831_20160901.csv")
data.head()

,Date,MSFT_Price,AAPL_Price,MSFT_Log_Price,AAPL_Log_Price,MSFT_Log_Return,AAPL_Log_Return
0,31-Aug-11,26.60,54.98,3.280911,4.006969,NaN,NaN
1,1-Sep-11,26.21,54.43,3.266141,3.996915,-0.014770,-0.010054
2,2-Sep-11,25.80,53.44,3.250374,3.978560,-0.015767,-0.018356
3,6-Sep-11,25.51,54.25,3.239071,3.993603,-0.011304,0.015043
4,7-Sep-11,26.00,54.85,3.258097,4.004602,0.019026,0.010999


In [3]:
data = data.dropna(axis=0)
data.head()

,Date,MSFT_Price,AAPL_Price,MSFT_Log_Price,AAPL_Log_Price,MSFT_Log_Return,AAPL_Log_Return
1,1-Sep-11,26.21,54.43,3.266141,3.996915,-0.014770,-0.010054
2,2-Sep-11,25.80,53.44,3.250374,3.978560,-0.015767,-0.018356
3,6-Sep-11,25.51,54.25,3.239071,3.993603,-0.011304,0.015043
4,7-Sep-11,26.00,54.85,3.258097,4.004602,0.019026,0.010999
5,8-Sep-11,26.22,54.88,3.266522,4.005149,0.008426,0.000547


The following function will estimate the EWMA of the returns 

$\mu_{s+\Delta} = \lambda\mu_s + (1-\lambda)X_s$

In [4]:
def EWMA_X(X, numb_initial_obs,lam = 0.97):
    """
    This function takes in a Series of returns - X 
    and generates the Exponentially Weighted Moving Average of the series. 
    It uses the initial M observations as the larger weighted term (multiplied by mu)
    """
    numb_mus = len(X)- numb_initial_obs
    mu = pd.Series(np.repeat(0.0,(numb_mus)))
    mu[0] = X[0:numb_initial_obs].mean()
    for s_delta in range(1,numb_mus):
        s = s_delta - 1 
        mu[s_delta] = lam * mu[s] + (1-lam)*X[(numb_initial_obs+s)]
        
    return mu

In [83]:
def EWMA_cov(returns, numb_initial_obs, lam = 0.97):
    """
    This function computes the EWMA covariance matrix for a 2x2 matrix. 
    We calculate the elements of the matrix individually. 
    We return a 3 x (numb remaining elements) data frame 
    """
    numb_covs = len(returns)- numb_initial_obs
    
    #Sigma11 = Sigma22 = Sigma12 = pd.Series(np.repeat(0.0,(numb_mus)))
    #Sigma11[0] = X1[0:numb_covs].var()
    #Sigma22[0] = X2[0:numb_covs].var()
    #Sigma12[0] = 
    Sigmas = {}
    Sigmas[0] = returns[0:numb_initial_obs].cov()
    
    for s_delta in range(1, numb_covs):
        s = s_delta -1 
        pt1 = (1-lam)*returns.iloc[(numb_initial_obs+s)].dot(returns.iloc[(numb_initial_obs+s)])
        #returns.iloc[(numb_initial_obs+s)]*returns.iloc[(numb_initial_obs+s)].transpose()
        Sigmas[s_delta] = lam * Sigmas[s] + pt1
    return Sigmas

In [84]:
MSFT_mu = EWMA_X(data["MSFT_Log_Return"], 500)
AAPL_mu = EWMA_X(data["AAPL_Log_Return"], 500)
mu = pd.DataFrame({"AAPL" : AAPL_mu,
                    "MSFT" : MSFT_mu})

This function finds the EWMA for a covariance matrix 

$\Sigma_{s+\Delta} = \lambda \Sigma_s +(1-\lambda)X_s X_s^T$ 

In [100]:
returns = pd.DataFrame({'MSFT': data['MSFT_Log_Return'],
                  'AAPL':data['AAPL_Log_Return']})
EWMA_cov1 = EWMA_cov(returns, 500)
EWMA_cov1[0]

,AAPL,MSFT
AAPL,0.000349,0.000070
MSFT,0.000070,0.000217


We will now compute the linearized loss operator knowing 
- Value of portfolio is 1,000,000
- Market cap of Microsoft is 448.77 million
- Market cap of Apple is 577.1 million 
- Linear Loss Operator is: $L_{t+\Delta} = c_t^T X_{t+\Delta}$   where $c_t =-V_t(w^1_t,w^2_t)$

In [86]:
ValPort = 1000000
MarketCapMSFT = 448.77
MarketCapAAPL = 577.1
WeightMSFT = MarketCapMSFT/ (MarketCapMSFT+MarketCapAAPL)
WeightAAPL = MarketCapAAPL/(MarketCapMSFT+MarketCapAAPL)
LinearLoss = -ValPort * ((WeightAAPL*data["AAPL_Log_Return"])
                       +(WeightMSFT*data["MSFT_Log_Return"]))
ct = pd.Series([-ValPort*WeightAAPL, -ValPort*WeightMSFT])

The Value at Risk of a portfolio characterized by the Linearized Loss operator is

$VaR(\alpha) = c_t^T\mu_{t+\Delta} + (c_t^T\Sigma_{t+\Delta}
   c_t)^(1/2)N^{-1}(\alpha)$
   
The K-day VaR is = 
$ VaR_{\alpha} (L'_{t+K\Delta}) \approx \sqrt(K)*VaR_{\alpha} (L'_{t+\Delta}) $

The regulatory charge is 

$ 3 \times VaR_{\alpha} (L'_{t+10\Delta}) $

In [95]:
last_element = len(EWMA_cov1) -2 
mu1 = mu.iloc[last_element]
Sigma1 = EWMA_cov1[140]

k = 20 
alpha = 0.05
N_alpha = norm.ppf(alpha)

Var = np.dot(mu1,ct)
Var2 = np.dot(np.dot(ct,Sigma1),ct.transpose())*0.5*N_alpha
np.sqrt(-Var2)

17401.113205678121

In [96]:
-np.sqrt(-Var2) + Var

-18782.324093807318

In [99]:
Sigma1

,AAPL,MSFT
AAPL,0.000370,0.000367
MSFT,0.000367,0.000369


In [45]:
np.random.multivariate_normal(mu1, Sigma1,5)

array([[ 0.00422671,  0.00652782],
       [ 0.0262212 ,  0.03001319],
       [-0.00648544, -0.01345049],
       [ 0.00520838,  0.00486635],
       [ 0.03070281,  0.03343353]])